In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm_notebook
import pandas as pd
import time


In [2]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.headless=True
chrome_options.add_experimental_option("detach", True)


In [3]:
def scroll_down(driver):
    last_element = None  
    while True:
        url_raw = driver.find_elements(By.CSS_SELECTOR, '.JobCard_JobCard__Tb7pI a') 
        if not url_raw: 
            break
        if last_element == url_raw[-1]:  
            return url_raw
        last_element = url_raw[-1]  
        ActionChains(driver).move_to_element(last_element).perform()
        time.sleep(1)  

In [4]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)  

# 원티드드 사이트로 이동합니다.
driver.get(url='https://www.wanted.co.kr/wdlist?country=kr&job_sort=job.latest_order&years=0&locations=all')
#직군 선택 
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/button/span/span[2]').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[1]/button[2]').click()
develope_oc = WebDriverWait(driver, 3).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "SelectItem_SelectItem__IcYKH"))
)
Occupation_list={'종료':0}
num=2
for button in develope_oc:
    try:
        occupation = WebDriverWait(button, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "wds-1c3y3mo"))
        )
        Occupation_list[occupation.text]=num
        num+=1
    except:
        pass
print(Occupation_list)
time.sleep(1)
while True:
    input_occupation=input('직군선택: ')
    if input_occupation == '종료':
        break
    occupation=Occupation_list[input_occupation]
    driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[2]/button[{occupation}]/span').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[3]/button[2]/span[2]').click()
time.sleep(1)
scroll_down(driver)
url_raw=scroll_down(driver)
url_list=[]
for url in url_raw:
    try:
        link = url.get_attribute("href")  
        url_list.append(link)
    except:
        pass
print(len(url_list))



{'종료': 0, '소프트웨어 엔지니어': 2, '서버 개발자': 3, '프론트엔드 개발자': 4, '웹 개발자': 5, '자바 개발자': 6, '파이썬 개발자': 7, '머신러닝 엔지니어': 8, 'C,C++ 개발자': 9, '데이터 엔지니어': 10, 'DevOps / 시스템 관리자': 11, 'Node.js 개발자': 12, '안드로이드 개발자': 13, '임베디드 개발자': 14, '시스템,네트워크 관리자': 15, 'QA,테스트 엔지니어': 16, '개발 매니저': 17, '보안 엔지니어': 18, 'iOS 개발자': 19, '데이터 사이언티스트': 20, '기술지원': 21, '하드웨어 엔지니어': 22, '프로덕트 매니저': 23, '빅데이터 엔지니어': 24, '웹 퍼블리셔': 25, '.NET 개발자': 26, '크로스플랫폼 앱 개발자': 27, 'PHP 개발자': 28, '블록체인 플랫폼 엔지니어': 29, '영상,음성 엔지니어': 30, 'DBA': 31, 'ERP전문가': 32, '그래픽스 엔지니어': 33, 'CTO,Chief Technology Officer': 34, '루비온레일즈 개발자': 35, 'BI 엔지니어': 36, 'VR 엔지니어': 37, 'CIO,Chief Information Officer': 38, 'RPA 엔지니어': 39, '테크니컬 라이터': 40}
61


In [5]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)
data_dict={}

for n in tqdm_notebook(range(len(url_list)),desc="진행률"):
    data_info={}
    Main_task, qualifications, Preferred_Qualifications=None, None, None
    driver.get(url_list[n])
    driver.implicitly_wait(2)
    try:
        see_more=driver.find_elements(By.CLASS_NAME, 'Button_Button__root__m1NGq.Button_Button__outlined__0HnEd.Button_Button__outlinedAssistive__JKDyz.Button_Button__outlinedSizeLarge__A_H8o.Button_Button__fullWidth__zAnDP')
        for button in see_more:
            button.click()
    except:
        pass
    title = driver.find_element(By.CLASS_NAME, 'wds-jtr30u').text
    location = driver.find_element(By.CLASS_NAME, 'JobHeader_JobHeader__Tools__Company__Info__yT4OD.wds-rgovpd').text
    explanation_things=driver.find_elements(By.CSS_SELECTOR, "div.JobDescription_JobDescription__paragraph__Lhegj h3.wds-1y0suvb")
    explanation_detail=driver.find_elements(By.CSS_SELECTOR, "div.JobDescription_JobDescription__paragraph__Lhegj span.wds-wcfcu3")
    for i in range(len(explanation_things)):
        if explanation_things[i].text == '주요업무':
            Main_task=explanation_detail[i].text
        elif explanation_things[i].text == '자격요건':
            qualifications=explanation_detail[i].text
        elif explanation_things[i].text == '우대사항':
            Preferred_Qualifications=explanation_detail[i].text
    try:
        skils = driver.find_elements(By.CSS_SELECTOR, 'li.SkillTagItem_SkillTagItem__K3B3t span.wds-1m3gvmz')
        skils = [skil.text for skil in skils]
    except:
        skils=None
    
    data_info['title']=title
    data_info['location']=location
    data_info['Main_task']=Main_task
    data_info['qualifications']=qualifications
    data_info['Preferred_Qualifications']=Preferred_Qualifications
    data_info['skills']=skils
    data_info['url']=url_list[n]
    
    data_dict[n]=data_info
    time.sleep(0.5)
    print(n,title)
    
result_df=pd.DataFrame.from_dict(data_dict, 'index')
result_df.to_csv('wanted.csv', encoding='utf-8-sig', index=False)
print('저장완료',len(data_dict))

C:\Users\User\AppData\Local\Temp\ipykernel_25748\1939611148.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(url_list)),desc="진행률"):


진행률:   0%|          | 0/61 [00:00<?, ?it/s]

0 [신입] 머신러닝 엔지니어 (LLM)
1 [카카오계열사]Prompt Engineer
2 Google Cloud Platform Solutions Engineer
3 코덱SW엔지니어 (화질분석/신입)
4 코덱SW엔지니어 (알고리즘개발/신입)
5 소프트웨어 엔지니어
6 Jr. ML Engineer(일반/전문연구요원)
7 딥러닝/머신러닝(AI) 개발자 [신입, 전문연구요원(보충역/전직) 가능
8 AI Engineer (병특 전문연구요원가능)
9 [이미지/비디오 생성형 AI] Research Engineer (전문연구요원)
10 Research Scientist
11 전문연구요원_AI 개발자 (병역 특례)
12 Operation / Digital Consultant
13 ML engineer/ AI researcher (전문연구요원 - 보충역, 전직자)
14 ML engineer/ AI researcher (Junior)
15 딥러닝 연구원 / 엔지니어 (전문연구요원 가능)
16 [AI연구소] AI연구원 - Computer Vision AI
17 신약개발 특허분석 인공지능 연구원
18 Computer Vision AI Engineer_Junior
19 Prompt Engineer
20 인공지능 음성 합성(TTS) 전문가
21 Physical AI Research Scientist (강화학습, 시뮬레이션)
22 [인턴] AI Team
23 NPU RTL 설계 엔지니어 (전문연구요원/신입)
24 [전문연구요원] Computer Vision Engineer
25 연구소 회로설계 및 HW 개발
26 AI 엔지니어(NLP/LLM)
27 인공지능(AI) 개발자
28 SW개발 AI 개발자
29 AI 엔지니어 (Stable diffusion / VITON)
30 딥러닝 엔지니어
31 머신러닝 엔지니어
32 딥러닝/머신러닝 분야 연구원 (인공지능 개발자)
33 CAD Engineer - 전문연구요원 가능
34 AI엔지니어
35 AI 최적화 Researcher
36 (전문연구요원 

In [6]:
result_df.head()

,title,location,Main_task,qualifications,Preferred_Qualifications,skills,url
0,[신입] 머신러닝 엔지니어 (LLM),서울 서초구,• 데이터를 분석하고 풀어야 할 문제를 정의합니다.\n• 동료들과 더 나은 LLM ...,• LLM(Fine tuning 포함) 또는 NLP 모델의 학습 경험이 있는 분\n...,• 다양한 분야의 데이터를 다룬 경험이 있는 분\n• AI Agent 활용 경험이 ...,[],https://www.wanted.co.kr/wd/264181
1,[카카오계열사]Prompt Engineer,서울 강남구,"• LLM 프롬프트 최적화 및 엔지니어링: 다양한 LLM(GPT, Claude, G...",• 해외여행 결격사유가 없는 분\n• LLM 및 NLP 실무 경험: LLM 또는 자...,ML 기반 서비스 개발 또는 백엔드 개발 경험\n• ML 기반 서비스 개발 경험: ...,"[Python, NLP]",https://www.wanted.co.kr/wd/264121
2,Google Cloud Platform Solutions Engineer,서울 강남구,• GCP 클라우드 아키텍처 설계 및 기술 컨설팅 수행\n• 고객사 클라우드 마이그...,• 클라우드 환경에서의 프로젝트 경험 보유 (토이 프로젝트 포함)\n• Genera...,• Generative AI 프로젝트 경험자\n• AI/ML 프로젝트 수행 경험 또...,[],https://www.wanted.co.kr/wd/263995
3,코덱SW엔지니어 (화질분석/신입),서울 강남구,"[다음 업무를 통해 목표를 달성하고 성장합니다]\n• 영상처리, 동영상 압축 알고리...","[이런분과 함께하고 싶어요]\n• 전자, 컴퓨터공학 학사 이상\n• C, Pytho...",[이런분이라면 더욱 환영해요]\n• 비디오 압축 혹은 영상 처리 관련 연구 수행 및...,"[Linux, Java, Python, C, C++]",https://www.wanted.co.kr/wd/174091
4,코덱SW엔지니어 (알고리즘개발/신입),서울 강남구,"[이런분과 함께하고 싶어요]\n• 영상처리, 동영상 압축 등 멀티미디어 처리 기술개발","[이런분이라면 더욱 환영해요]\n• 전자, 컴퓨터공학 학사 이상\n• C/C++ 능...",[더 몰입하실 수 있도록 도와드려요]\n• 비디오 압축 혹은 영상 처리 관련 연구 ...,"[Linux, Java, Python, C, C++]",https://www.wanted.co.kr/wd/174088
